# Importando as Bibliotecas e Examinando os Df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Importando e visualizando meu dataframe
df = pd.read_csv('Power_Plant.csv')

print(df.head(), end="\n\n")
print(df.info())
# PE = Y

In [ ]:
# Renomeando as colunas para melhor entendimento dos dados
df = df.rename(columns={'AT': 'Temperatura', 'V': 'Pressão do Ar','RH': 'Umidade do Ar','AP': 'Pressão ATM','PE': 'Energia Produzida',})
df.head()

In [ ]:
# Análisando a correlação das demais colunas com a 'Energia Produzida'
corr = df.corr()
corr['Energia Produzida']

In [ ]:
# Mostrando a distribuição da energia produzida no df
sns.displot(df['Energia Produzida'], kde=True, color='green')
plt.title('Distribuição da Energia Produzida')
plt.show()

In [ ]:
# Mostrando características de cada coluna que talvez possam explicar a energia produzida
sns.pairplot(df, y_vars='Energia Produzida', x_vars=['Temperatura','Pressão do Ar','Pressão ATM','Umidade do Ar'] )

# Treinando os Modelos

In [ ]:
# Definindo X e y e aplicando o split
y = df['Energia Produzida']
X = df.drop(columns='Energia Produzida')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=250)

In [ ]:
# Dataframe de treino pra usar na fórmula
df_train = pd.DataFrame(data= X_train)
df_train['Energia Produzida'] = y_train

In [ ]:
# Adicionando uma constante
X_train = sm.add_constant(X_train)
X_train

In [ ]:
X_train.columns

In [ ]:
# Criando alguns modelos de regreção para teste 

# Com todas as colunas
modelo_1 = sm.OLS(y_train,
                  X_train[['const', 'Temperatura', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']]).fit()
# Sem as colunas Temperatura e Pressão do Ar
modelo_2 = sm.OLS(y_train,
                  X_train[['const', 'Pressão ATM', 'Umidade do Ar']]).fit()
# Sem a coluna Temperatura
modelo_3 = sm.OLS(y_train,
                  X_train[['const', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']]).fit()
# Sem a coluna Pressão do Ar
modelo_4 = sm.OLS(y_train,
                  X_train[['const', 'Temperatura', 'Pressão ATM', 'Umidade do Ar']]).fit()

In [ ]:
print(modelo_1.summary())

In [ ]:
print(modelo_2.summary())

In [ ]:
print(modelo_3.summary())

In [ ]:
print(modelo_4.summary())

In [ ]:
# R2 de cada modelo
print("R²")
print("Modelo 1: ", modelo_1.rsquared)
print("Modelo 2: ", modelo_2.rsquared)
print("Modelo 3: ", modelo_3.rsquared)
print("Modelo 4: ", modelo_4.rsquared)

In [ ]:
#Quantos parametros estão em cada modelo
print(len(modelo_1.params))
print(len(modelo_2.params))
print(len(modelo_3.params))
print(len(modelo_4.params))

# Testando o Modelo Inicialmente Escolhido

*   Eu escolhi o modelo 1 até esse momento, pois ele possui mais parametros e R2 foi o maior.
*   Entreteando há um multicolinearidade a ser investigada.

In [ ]:
modelo_1.params

In [ ]:
# Adicionando uma constante em X_test
X_test = sm.add_constant(X_test)

In [ ]:
# Prevendo com o modelo 1
predict_1 = modelo_1.predict(X_test[['const', 'Temperatura', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']])

In [ ]:
# Qual o r² da previsão?
modelo_1.rsquared

In [ ]:
# Qual o R² do treino?
print("R²: ", r2_score(y_test, predict_1))

In [ ]:
# Dados que quero obter previsão
novo_dado = pd.DataFrame({ 'const': [1],
                             'Temperatura': [18.5],
                              'Pressão do Ar': [32.1],
                              'Pressão ATM': [1008.20],
                              'Umidade do Ar':[70.99]
})

In [ ]:
modelo_1.predict(novo_dado)[0]

# Investigado a Multicolimulticolinearidade e Escolhendo o Modelo Final a ser Usado



In [ ]:
explicativas_1 = ['const', 'Temperatura', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']

explicativas_2 = ['const', 'Pressão ATM', 'Umidade do Ar']

explicativas_3 = ['const', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']

explicativas_4 = ['const', 'Temperatura', 'Pressão ATM', 'Umidade do Ar']

In [ ]:
# VIF 1
vif_1 = pd.DataFrame()
vif_1['variavel'] = explicativas_1

vif_1["vif"] = [ variance_inflation_factor(X_train[explicativas_1], i ) for i in range(len(explicativas_1))]

vif_1

In [ ]:
# VIF 2
vif_2 = pd.DataFrame()
vif_2['variavel'] = explicativas_2

vif_2["vif"] = [ variance_inflation_factor(X_train[explicativas_2], i ) for i in range(len(explicativas_2))]

vif_2

In [ ]:
# VIF 3
vif_3 = pd.DataFrame()
vif_3['variavel'] = explicativas_3

vif_3["vif"] = [ variance_inflation_factor(X_train[explicativas_3], i ) for i in range(len(explicativas_3))]

vif_3

In [ ]:
# VIF 4
vif_4 = pd.DataFrame()
vif_4['variavel'] = explicativas_4

vif_4["vif"] = [ variance_inflation_factor(X_train[explicativas_4], i ) for i in range(len(explicativas_4))]

vif_4

*   Considerando a Multicolimulticolinearidade apresentada modelo 1, o melhor modelo a se usar seria o 2 ou 3 ou 4.
*   Devido quantidade de parametros do modelo 3 ser maior que os outros, eu escolho ele como melhor opção.


In [ ]:
# Treinando o modelo 3

predict_3 = modelo_3.predict(X_test[['const', 'Pressão do Ar', 'Pressão ATM', 'Umidade do Ar']])

In [ ]:
# Qual o r² da previsão?
modelo_3.rsquared

In [ ]:
# Qual o R² do treino?
print("R²: ", r2_score(y_test, predict_3))

In [ ]:
# Dados que quero obter previsão sem a temperatura para o modelo 3
novo_dado_3 = pd.DataFrame({ 'const': [1],
                              'Pressão do Ar': [32.1],
                              'Pressão ATM': [1008.20],
                              'Umidade do Ar':[70.99]
})

In [ ]:
#Comparando os 2 modelos treinados
print("Valores de Energia Obtida no Modelo Inicial 1:", modelo_1.predict(novo_dado)[0])
print("Valores de Energia Obtida no Modelo Final 3:", modelo_3.predict(novo_dado_3)[0])

# Análise dos Resíduos do Modelo Final

In [ ]:
# Prevendo os valores do treino X_traind do modelo 3
y_previsto_train = modelo_3.predict(X_train[explicativas_3])

In [ ]:
# Fazendo um scatter plot entre y_previsto e y_train

fig = px.scatter(x= y_previsto_train    , y= y_train,
                 title='Previsão X Real',
                 labels={'x':'Energia Prevista','y':'Energia real'})
fig.show()
print("Como visto no gráfico, o modelo está bem próximo da realidade")